`requirements/gpu.txt`

In [ ]:
import numpy as np, jsonpickle
from qiskit_aer.noise import NoiseModel
from qiskit_aer import AerSimulator
from qiskit_experiments.library import QuantumVolume

In [2]:
def load_noisy_simulator_from_calibration(path):
    with open(path, "r") as f:
        payload = jsonpickle.decode(f.read())

    noise_model = NoiseModel.from_dict(payload["noise_model"])
    coupling_map = payload.get("coupling_map")

    coupling_map = payload.get("coupling_map")

    return AerSimulator(
        noise_model=noise_model,
        coupling_map=coupling_map,
        basis_gates=noise_model.basis_gates,
        method="automatic",
        device="GPU",
    )

In [4]:
def run_qv_for_subset_1(calibration_path, subset, shots=2000, trials=100, seed=42):
    backend = load_noisy_simulator_from_calibration(calibration_path)

    ideal_backend = AerSimulator(method="statevector", device="GPU")

    exp = QuantumVolume(
        physical_qubits=subset,
        trials=trials,
        seed=seed,
        simulation_backend=ideal_backend,
    )

    exp.set_run_options(shots=shots)

    exp.set_transpile_options(
        layout_method="trivial",
        routing_method="basic",
        optimization_level=0,
        seed_transpiler=seed,
        num_processes=1,
    )

    return exp.run(backend).block_for_results()

In [17]:
import quantum_volume as qv

def run_qv_for_subset_2(calibration_path, subset, shots=2000, trials=100, seed=42):
    backend = load_noisy_simulator_from_calibration(calibration_path)
    
    return qv.quantum_volume(
        N=len(subset),
        k=trials,
        n=shots,
        backend=backend,
        subset=subset,
        seed=seed,
    )

In [10]:
exp_data_1 = run_qv_for_subset_1(
    "calibrations/ibm_marrakesh/20260129_101824.json",
    subset=[0, 1, 2, 3, 4, 5, 6],
)

exp_data_2 = run_qv_for_subset_2(
    "calibrations/ibm_marrakesh/20260129_101824.json",
    subset=[0, 1, 2, 3, 4, 5, 6],
)

/tmp/ipykernel_24803/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])
/tmp/ipykernel_24803/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])


In [12]:
exp_data_1 = run_qv_for_subset_1(
    "calibrations/ibm_marrakesh/20260129_101824.json",
    subset=[0, 1, 2, 3, 4],
)

/tmp/ipykernel_2830/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])


In [15]:
print(exp_data_1.analysis_results())

[AnalysisResult(name=mean_HOP, value=0.8310599999999999+/-0.03746989143298924, device_components=[<Qubit(Q0)>, <Qubit(Q1)>, <Qubit(Q2)>, <Qubit(Q3)>, <Qubit(Q4)>], experiment_id=4fc2e4f4-b92b-44eb-b38c-8c559ceb9d7a, result_id=e294992e-fb5d-4f90-834e-d9f91140fe79, chisq=None, quality=good, verified=False, extra={'HOPs': [0.8035, 0.8375, 0.8655, 0.8565, 0.8515, 0.777, 0.8305, 0.842, 0.8475, 0.862, 0.825, 0.79, 0.875, 0.795, 0.863, 0.887, 0.823, 0.799, 0.8745, 0.869, 0.7945, 0.812, 0.838, 0.78, 0.8695, 0.8325, 0.834, 0.8185, 0.898, 0.821, 0.8255, 0.729, 0.9025, 0.81, 0.865, 0.821, 0.851, 0.8525, 0.7435, 0.8545, 0.8415, 0.764, 0.8405, 0.9035, 0.8135, 0.8825, 0.8655, 0.82, 0.9075, 0.8125, 0.8655, 0.832, 0.8045, 0.8155, 0.8355, 0.832, 0.793, 0.8025, 0.8235, 0.831, 0.8425, 0.8025, 0.8695, 0.891, 0.8535, 0.8615, 0.835, 0.837, 0.845, 0.826, 0.8365, 0.81, 0.8215, 0.816, 0.7995, 0.854, 0.809, 0.8035, 0.869, 0.778, 0.8075, 0.9605, 0.841, 0.8115, 0.751, 0.852, 0.805, 0.8635, 0.8285, 0.7925, 0.7165,

/tmp/ipykernel_2830/3295461704.py:1: DeprecationWarning: Leaving `dataframe` unset or setting it to `False` for `ExperimentData.analysis_results` is deprecated as of qiskit-experiments 0.9.0. Future releases may change the default to `True` and remove the option to set the value to `False`.
  print(exp_data_1.analysis_results())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_analysis_value(exp_data, name: str):
    """
    Robust extraction of an analysis result value by name across qiskit-experiments versions.
    Tries dataframe=True first to avoid the deprecation warning.
    """
    try:
        ars = exp_data.analysis_results(dataframe=True)
        # dataframe=True returns a pandas DF in newer versions
        # with columns like ["name", "value", ...]
        if hasattr(ars, "loc"):
            row = ars.loc[ars["name"] == name]
            if len(row) == 0:
                raise KeyError(name)
            return row["value"].iloc[0]
    except TypeError:
        # older versions don't have dataframe kwarg
        pass

    # fallback: list of AnalysisResult objects
    ars = exp_data.analysis_results()
    for ar in ars:
        if ar.name == name:
            return ar.value
    raise KeyError(f"Analysis result '{name}' not found. Available: {[ar.name for ar in ars]}")

def mean_from_uncertainties_var(uvar):
    """uvar is uncertainties.core.Variable: use .n for nominal."""
    return float(getattr(uvar, "n", uvar))

def run_compare_mean_hop_by_N(calibration_path, Ns=(2, 3, 4), shots=2000, trials=100, seed=42):
    qiskit_means = []
    qiskit_sigmas = []  # if available
    home_means = []
    home_sigmas = []

    for N in Ns:
        subset = list(range(N))

        # --- Qiskit Experiments run ---
        exp_data_1 = run_qv_for_subset_1(
            calibration_path,
            subset=subset,
            shots=shots,
            trials=trials,
            seed=seed,
        )

        mean_HOP_var = get_analysis_value(exp_data_1, "mean_HOP")  # uncertainties Variable
        mu1 = float(mean_HOP_var.n)
        s1 = float(mean_HOP_var.s)

        # --- Homemade run ---
        out2 = run_qv_for_subset_2(
            calibration_path,
            subset=subset,
            shots=shots,
            trials=trials,
            seed=seed,
        )
        hops2 = np.array(out2["individual_hops"], dtype=float)
        mu2 = float(np.mean(hops2))
        # use standard error of mean as comparable "uncertainty"
        s2 = float(np.std(hops2, ddof=1) / np.sqrt(len(hops2))) if len(hops2) > 1 else 0.0

        qiskit_means.append(mu1); qiskit_sigmas.append(s1)
        home_means.append(mu2);   home_sigmas.append(s2)

        print(f"N={N} subset={subset}")
        print(f"  Qiskit mean_HOP = {mu1:.4f} ± {s1:.4f}")
        print(f"  Home   mean_HOP = {mu2:.4f} ± {s2:.4f} (SEM)")
        print()

    # --- Plot mean_HOP vs N with error bars ---
    Ns = list(Ns)
    plt.figure(figsize=(7,4))
    plt.errorbar(Ns, qiskit_means, yerr=[2*s for s in qiskit_sigmas], fmt="o-", capsize=6, label="Qiskit Experiments (±2σ)")
    plt.errorbar(Ns, home_means,   yerr=[2*s for s in home_sigmas],   fmt="s--", capsize=6, label="Homemade (±2·SEM)")

    plt.axhline(2/3, linestyle=":", linewidth=2, label="2/3 threshold")
    plt.xticks(Ns)
    plt.xlabel("N (width/depth)")
    plt.ylabel("Final mean HOP")
    plt.title("Final mean HOP by N (2..4)")
    plt.grid(alpha=0.3)
    plt.legend()
    plt.show()

    return {
        "N": Ns,
        "qiskit_mean_HOP": qiskit_means,
        "qiskit_sigma": qiskit_sigmas,
        "home_mean_HOP": home_means,
        "home_sem": home_sigmas,
    }

# ---- run it ----
summary = run_compare_mean_hop_by_N(
    "calibrations/ibm_marrakesh/20260129_101824.json",
    Ns=(2,3,4,5,6,7),
    shots=2000,
    trials=100,
    seed=42
)


/tmp/ipykernel_2830/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])
/tmp/ipykernel_2830/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])


N=2 subset=[0, 1]
  Qiskit mean_HOP = 0.7897 ± 0.0408
  Home   mean_HOP = 0.7986 ± 0.0100 (SEM)



/tmp/ipykernel_2830/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])
/tmp/ipykernel_2830/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])


N=3 subset=[0, 1, 2]
  Qiskit mean_HOP = 0.7075 ± 0.0455
  Home   mean_HOP = 0.8239 ± 0.0082 (SEM)



/tmp/ipykernel_2830/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])
/tmp/ipykernel_2830/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])


N=4 subset=[0, 1, 2, 3]
  Qiskit mean_HOP = 0.8189 ± 0.0385
  Home   mean_HOP = 0.8098 ± 0.0052 (SEM)



/tmp/ipykernel_2830/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])
